# Simple Docstrum
## Sources
- [_The Document Spectrum for Page Layout Analysis_](https://ieeexplore.ieee.org/document/244677), Lawrence O’Gorman
- https://inside.mines.edu/~whoff/courses/EENG510/projects/2015/Hoch.pdf
- https://en.wikipedia.org/wiki/Document_layout_analysis#Example_of_a_bottom_up_approach


- https://github.com/UglyToad/PdfPig

## Steps
0. Open pdf document, extract words and preprocess
1. Estimate within-line and between-line spacing
2. Get lines
3. Get Paragraphs

In [22]:
//#r "nuget:PdfPig"
#r "D:\VS2017\source\repos\BobLd\PdfPig\src\UglyToad.PdfPig.DocumentLayoutAnalysis\bin\Debug\netstandard2.0\UglyToad.PdfPig.dll"
#r "D:\VS2017\source\repos\BobLd\PdfPig\src\UglyToad.PdfPig.DocumentLayoutAnalysis\bin\Debug\netstandard2.0\UglyToad.PdfPig.Core.dll"
#r "D:\VS2017\source\repos\BobLd\PdfPig\src\UglyToad.PdfPig.DocumentLayoutAnalysis\bin\Debug\netstandard2.0\UglyToad.PdfPig.Tokens.dll"
#r "D:\VS2017\source\repos\BobLd\PdfPig\src\UglyToad.PdfPig.DocumentLayoutAnalysis\bin\Debug\netstandard2.0\UglyToad.PdfPig.Tokenization.dll"
#r "D:\VS2017\source\repos\BobLd\PdfPig\src\UglyToad.PdfPig.DocumentLayoutAnalysis\bin\Debug\netstandard2.0\UglyToad.PdfPig.Fonts.dll"
#r "D:\VS2017\source\repos\BobLd\PdfPig\src\UglyToad.PdfPig.DocumentLayoutAnalysis\bin\Debug\netstandard2.0\UglyToad.PdfPig.DocumentLayoutAnalysis.dll"
    
using UglyToad.PdfPig;
using UglyToad.PdfPig.Content;
using UglyToad.PdfPig.Core;
using UglyToad.PdfPig.DocumentLayoutAnalysis;
using UglyToad.PdfPig.DocumentLayoutAnalysis.WordExtractor;
using XPlot.Plotly;
using System.IO;
using System.Linq;

Invalid: Command: #r D:\VS2017\source\repos\BobLd\PdfPig\src\UglyToad.PdfPig.DocumentLayoutAnalysis\bin\Debug\netstandard2.0\UglyToad.PdfPig.dllInvalid: Command: #r D:\VS2017\source\repos\BobLd\PdfPig\src\UglyToad.PdfPig.DocumentLayoutAnalysis\bin\Debug\netstandard2.0\UglyToad.PdfPig.Core.dllInvalid: Command: #r D:\VS2017\source\repos\BobLd\PdfPig\src\UglyToad.PdfPig.DocumentLayoutAnalysis\bin\Debug\netstandard2.0\UglyToad.PdfPig.Tokens.dllInvalid: Command: #r D:\VS2017\source\repos\BobLd\PdfPig\src\UglyToad.PdfPig.DocumentLayoutAnalysis\bin\Debug\netstandard2.0\UglyToad.PdfPig.Tokenization.dllInvalid: Command: #r D:\VS2017\source\repos\BobLd\PdfPig\src\UglyToad.PdfPig.DocumentLayoutAnalysis\bin\Debug\netstandard2.0\UglyToad.PdfPig.Fonts.dllInvalid: Command: #r D:\VS2017\source\repos\BobLd\PdfPig\src\UglyToad.PdfPig.DocumentLayoutAnalysis\bin\Debug\netstandard2.0\UglyToad.PdfPig.DocumentLayoutAnalysis.dll

### 0. Open pdf document, extract words and preprocess

In [23]:
string pdfName = "Random 2 Columns Lists Hyph.pdf";
//string pdfName = "Random 2 Columns Lists Hyph - Justified.pdf";
int pageNo = 1;

List<Word> wordsRaw = new List<Word>();
double width = 0;
double height = 0;

using (PdfDocument document = PdfDocument.Open(pdfName))
{
    var page = document.GetPage(pageNo);
    width = page.Width;
    height = page.Height;
    wordsRaw = page.GetWords(NearestNeighbourWordExtractor.Instance).ToList();
}

var words = new List<Word>();
// only keep non space words
foreach (var word in wordsRaw)
{
    if (string.IsNullOrWhiteSpace(word.Text.Trim())) continue;
    words.Add(word);
}

#### Plot words

In [36]:
var graphs = new List<Graph.Scatter>();
graphs.Add(new Graph.Scatter()
{
    x = new[] { 0, 0, width, width, 0 },
    y = new[] { 0, height, height, 0, 0 },
    mode = "line",
    name = "",
    text = "page",
    marker = new Graph.Marker() { color = "brown" }
});

foreach (var w in words)
{
    graphs.Add(new Graph.Scatter()
    {
        x = new[] 
        { 
            w.BoundingBox.BottomLeft.X, 
            w.BoundingBox.BottomRight.X, 
            w.BoundingBox.TopRight.X, 
            w.BoundingBox.TopLeft.X,
            w.BoundingBox.BottomLeft.X 
        },
        y = new[]
        { 
            w.BoundingBox.BottomLeft.Y,
            w.BoundingBox.BottomRight.Y,
            w.BoundingBox.TopRight.Y,
            w.BoundingBox.TopLeft.Y,
            w.BoundingBox.BottomLeft.Y
        },
        mode = "lines",
        marker = new Graph.Marker() { color = "black" }
    });
}

var topLefts = new Graph.Scatter()
{
    x = words.Select(w => w.BoundingBox.TopLeft.X),
    y = words.Select(w => w.BoundingBox.TopLeft.Y),
    name = "",
    mode = "markers",
    text = words.Select(w => w.Text + " (tl)"),
    marker = new Graph.Marker() { color = "blue", size = 4 }
};
graphs.Add(topLefts);

var bottomLefts = new Graph.Scatter()
{
    x = words.Select(w => w.BoundingBox.BottomLeft.X),
    y = words.Select(w => w.BoundingBox.BottomLeft.Y),
    name = "",
    mode = "markers",
    text = words.Select(w => w.Text + " (bl)"),
    marker = new Graph.Marker() { color = "yellow", size = 4 }
};
graphs.Add(bottomLefts);

var bottomRights = new Graph.Scatter()
{
    x = words.Select(w => w.BoundingBox.BottomRight.X),
    y = words.Select(w => w.BoundingBox.BottomRight.Y),
    name = "",
    mode = "markers",
    text = words.Select(w => w.Text + " (br)"),
    marker = new Graph.Marker() { color = "red", size = 4 }

};
graphs.Add(bottomRights);

var chart = Chart.Plot(graphs.ToArray());
chart.WithLayout(new Layout.Layout() 
                 { 
                     title = pdfName + " - Words",
                     showlegend = false,
                     hovermode = "closest",
                     width = width,
                     height = height,
                     xaxis = new Graph.Xaxis() {  range = new[] { 0, width } },
                     yaxis = new Graph.Yaxis() {  range = new[] { 0, height } },
                 });
chart.WithXTitle("X");
chart.WithYTitle("Y");
chart.Width = (int)width;
chart.Height = (int)height;
display(chart);

### 1.  Estimate within-line and between-line spacing

In [25]:
/// <summary>
/// The bounds for the angle between two words for them to have a certain type of relationship.
/// </summary>
public struct AngleBounds
{
    /// <summary>
    /// The lower bound in degrees.
    /// </summary>
    public double Lower { get; }

    /// <summary>
    /// The upper bound in degrees.
    /// </summary>
    public double Upper { get; }

    /// <summary>
    /// Create a new <see cref="AngleBounds"/>.
    /// </summary>
    public AngleBounds(double lowerBound, double upperBound)
    {
        Lower = lowerBound;
        Upper = upperBound;
    }

    /// <summary>
    /// Whether the bounds contain the angle.
    /// </summary>
    public bool Contains(double angle)
    {
        return angle >= Lower && angle <= Upper;
    }
}

In [26]:
AngleBounds withinLine = new AngleBounds(-30, 30);
AngleBounds betweenLine = new AngleBounds(-135, -45);

In [27]:
/// <summary>
/// Get the average distance value of the peak bucket of the histogram.
/// </summary>
/// <param name="distances">The set of distances to average.</param>
/// <param name="binLength"></param>
private static double? GetPeakAverageDistance(IEnumerable<double> distances, int binLength = 1)
{
    var max = (int)Math.Ceiling(distances.Max());
    binLength = binLength > max ? max : binLength;
    var bins = Enumerable.Range(0, (int)Math.Ceiling(max / (double)binLength) + 1)
        .Select(x => x * binLength)
        .ToDictionary(x => x, _ => new List<double>());

    foreach (var distance in distances)
    {
        var key = bins.Keys.ElementAt((int)Math.Floor(distance / binLength));
        bins[key].Add(distance);
    }

    display(bins.ToDictionary(x => x.Key, x => Math.Round(x.Value.Count() / (double)distances.Count() * 100, 5)));
    
    var best = default(List<double>);
    foreach (var bin in bins)
    {
        if (best == null || bin.Value.Count > best.Count)
        {
            best = bin.Value;
        }
    }

    return best?.Average();
}

#### 1.1 Within-line (between words) spacing
For within-line spacing, the distance between the bottom left (in yellow) and bottom right (in red) point will be computed

In [28]:
KdTree<Word> kdTreeWL = new KdTree<Word>(words, w => w.BoundingBox.BottomLeft);

// Create distribution
var withinLineDistList = new List<double>();
foreach (var candidateW in words)
{
    // 1.1.1 Find the 2 closest neighbours words to the candidate, using weighted euclidean distance that gives more weight to y axis.
    // Words on the same line (same y) have more chance to be neighbours
    // NB: might need to use a different method with rotated words
    foreach (var n in kdTreeWL.FindNearestNeighbours(candidateW, 2, w => w.BoundingBox.BottomRight, 
                                                     (p1, p2) => Distances.WeightedEuclidean(p1, p2, 0.5)))
    {
        // 1.1.2 Check if the neighbour word is within the angle of the candidate
        var angle = Distances.Angle(candidateW.BoundingBox.BottomRight, n.Item1.BoundingBox.BottomLeft);
        if (withinLine.Contains(angle))
        {
            // 1.1.3 Compute the horizontal (within-line) distance between the candidate 
            // and the neighbour and add it to the within-line distances list
            withinLineDistList.Add(Distances.Horizontal(candidateW.BoundingBox.BottomRight, n.Item1.BoundingBox.BottomLeft));
        }
    }
}

// Compute average peak value of distribution
int wlBinSize = 10;
var wlDist = GetPeakAverageDistance(withinLineDistList, wlBinSize).Value;

// Plot histogram
var histoWl = new Graph.Histogram() 
{ 
    x = withinLineDistList, 
    xbins = new Graph.Xbins() 
    { 
        start = 0,
        end = Math.Ceiling(withinLineDistList.Max()),
        size = wlBinSize
    },
    text = "",
    name = "",
    histnorm = "percent"
};

var lineWlAvg = new Graph.Scatter() 
{ 
    x = new[] { wlDist },
    y = new[] { 0 },
    mode = "markers",
    name = "wl distance",
    marker = new Graph.Marker() 
    { 
        color = "red",
        size = 10
    }
};

var plotWl = Chart.Plot(new Graph.Trace[] { histoWl, lineWlAvg });
plotWl.WithLayout(new Layout.Layout() { title = "Distribution of within-line distances", showlegend = false, hovermode = "closest" });
plotWl.WithXTitle("distance");
plotWl.WithYTitle("%");
display(plotWl);

key,value
0,84.61538
10,11.1293
20,3.27332
30,0.491
40,0.32733
50,0.16367
60,0


#### 1.2 Between-line spacing
For between-line spacing, the distance between the top left (blue) and bottom left (yellow) point will be computed

In [29]:
KdTree<Word> kdTreeBL = new KdTree<Word>(words, w => w.BoundingBox.TopLeft);

// Create distribution
var betweenLineDistList = new List<double>();
foreach (var candidateW in words)
{
    // 1.2.1 Find the 2 closest neighbours words to the candidate, using weighted euclidean distance that gives more weight to x axis.
    // Words on the same 'column' (same x) have more chance to be neighbours
    // NB: might need to use a different method with rotated words
    foreach (var n in kdTreeBL.FindNearestNeighbours(candidateW, 2, w => w.BoundingBox.BottomLeft, (p1, p2) => Distances.WeightedEuclidean(p1, p2, 2)))
    {
        // 1.2.2 Check if the candidate words is within the angle
        var angle = Distances.Angle(candidateW.BoundingBox.Centroid, n.Item1.BoundingBox.Centroid);
        if (betweenLine.Contains(angle))
        {
            // 1.2.3 Compute the vertical (between-line) distance between the candidate 
            // and the neighbour and add it to the between-line distances list
            betweenLineDistList.Add(Distances.Vertical(candidateW.BoundingBox.BottomLeft, n.Item1.BoundingBox.TopLeft));
        }
    }
}

// Compute average peak value of distribution
int blBinSize = 10;
var blDist = GetPeakAverageDistance(betweenLineDistList, blBinSize).Value;

// Plot histogram``
var histoBl = new Graph.Histogram() 
{ 
    x = betweenLineDistList, 
    xbins = new Graph.Xbins() 
    { 
        start = 0,
        end = Math.Ceiling(betweenLineDistList.Max()),
        size = blBinSize
    },
    text = "",
    name = "",
    histnorm = "percent" 
};

var lineBlAvg = new Graph.Scatter() 
{ 
    x = new[] { blDist },
    y = new[] { 0 },
    mode = "markers",
    name = "bl distance",
    marker = new Graph.Marker() 
    { 
        color = "red", 
        size = 10 
    } 
};

var plotBl = Chart.Plot(new Graph.Trace[] { histoBl, lineBlAvg });
plotBl.WithLayout(new Layout.Layout() { title = "Distribution of between-line distances", showlegend = false, hovermode = "closest" });
plotBl.WithXTitle("distance");
plotBl.WithYTitle("%");
display(plotBl);

key,value
0,74.21875
10,5.46875
20,19.53125
30,0.58594
40,0.19531
50,0


In [35]:
double maxWlDist = 3.0 * wlDist; // Math.Min(3.0 * wlDist, 1.4142 * blDist); sqrt(2) * blDist
double maxBlDist = blDist * 1.3;
display("maxWlDist=" + Math.Round(maxWlDist, 3));
display("maxBlDist=" + Math.Round( maxBlDist,3));

maxWlDist=7.578

maxBlDist=9.965

### 2. Get lines

In [31]:
var lines = new List<TextLine>();
static double Angle(PdfPoint point1, PdfPoint point2)
{
    var angle = Distances.Angle(point1, point2);
    // Angle is kept at max 90 degree to handle overlapping words
    return angle <= 90 ? angle : 180 - angle;
}

var groupedIndexes = Clustering.NearestNeighbours(
    words,   // Group words into lines.
    2,   // Use 2 candidate neighbours words when trying to build the line.
    Distances.Euclidean,   // Use euclidian distance.
    (_, __) => maxWlDist,   // The maximum distance between 2 words in a line is 'maxWlDist'.
    pivot => pivot.BoundingBox.BottomRight,   // The distance will be computed between the pivot word's bottom right point and
    candidate => candidate.BoundingBox.BottomLeft,   // the candidate neighbour word's bottom left point.
    _ => true,   // No filter applied on the pivot word (always true).
    (pivot, candidate) => withinLine.Contains(   // The angle between the pivot word's bottom right point and
        Angle(pivot.BoundingBox.BottomRight,   // the candidate neighbour word's bottom left point should be
              candidate.BoundingBox.BottomLeft) - pivot.BoundingBox.Rotation), // inside the 'withinLine' angle.
    -1).ToList();   // Max degree of parallelism.

for (var a = 0; a < groupedIndexes.Count; a++)
{
    lines.Add(new TextLine(groupedIndexes[a].Select(i => words[i]).ToList()));
}

#### Plot lines

In [37]:
var graphs = new List<Graph.Scatter>();
graphs.Add(new Graph.Scatter()
{
    x = new[] { 0, 0, width, width, 0 },
    y = new[] { 0, height, height, 0, 0 },
    mode = "line",
    name = "",
    text = "page",
    marker = new Graph.Marker() { color = "brown" }
});

foreach (var p in lines)
{
    graphs.Add(new Graph.Scatter()
    {
        x = new[]
        {
            p.BoundingBox.BottomLeft.X, 
            p.BoundingBox.BottomRight.X, 
            p.BoundingBox.TopRight.X, 
            p.BoundingBox.TopLeft.X,
            p.BoundingBox.BottomLeft.X 
        },
        y = new[]
        {
            p.BoundingBox.BottomLeft.Y,
            p.BoundingBox.BottomRight.Y,
            p.BoundingBox.TopRight.Y,
            p.BoundingBox.TopLeft.Y,
            p.BoundingBox.BottomLeft.Y
        },
        text = p.Text.Count() <= 25 ? p.Text : string.Join("", p.Text.Take(25)) + "...",
        mode = "lines",
        marker = new Graph.Marker() { color = "red" }
    });
}

var chart = Chart.Plot(graphs.ToArray());
chart.WithLayout(new Layout.Layout() 
                 { 
                     title = pdfName + " - Lines",
                     showlegend = false,
                     hovermode = "closest",
                     width = width,
                     height = height,
                     xaxis = new Graph.Xaxis() {  range = new[] { 0, width } },
                     yaxis = new Graph.Yaxis() {  range = new[] { 0, height } },
                 });
chart.WithXTitle("X");
chart.WithYTitle("Y");
chart.Width = (int)width;
chart.Height = (int)height;
display(chart);

### 3. Get paragraphs blocks

In [33]:
var paragraphs = new List<TextBlock>();

// We want to measure the distance between two lines using the following method:
// - We check if two lines are overlapping horizontally.
// - If they are overlapping, we compute the middle point (new X coordinate) of the overlapping area.
// - We finally compute the Euclidean distance between these two middle points.
// - If the two lines are not overlapping, the distance is set to the max distance.

double euclidianOverlappingMiddleDistance(PdfLine l1, PdfLine l2)
{
    var left = Math.Max(l1.Point1.X, l2.Point1.X);
    var d = (Math.Min(l1.Point2.X, l2.Point2.X) - left);

    if (d < 0) return double.MaxValue; // not overlapping -> max distance
    return Distances.Euclidean(new PdfPoint(left + (d / 2.0), l1.Point1.Y),
                               new PdfPoint(left + (d / 2.0), l2.Point1.Y));
}

var groupedIndexes = Clustering.NearestNeighbours(
    lines,   // Group lines into paragraphs / blocks.
       // Use 1 candidate neighbour line when trying to build the paragraph.
    euclidianOverlappingMiddleDistance,   // Use euclidian overlapping middle distance.
    (_, __) => maxBlDist,   // The maximum distance between 2 lines in a paragraph is 'maxBlDist'.
    pivot => new PdfLine(pivot.BoundingBox.BottomLeft,   // The distance will be computed between the pivot bottom line and 
                         pivot.BoundingBox.BottomRight),   // 
    candidate => new PdfLine(candidate.BoundingBox.TopLeft,   // the candidate neighbour top line.
                             candidate.BoundingBox.TopRight),
    _ => true,   // No filter applied on the pivot line (always true).
    (_, __) => true,   // No other filter applied than the max dist (always true).
    -1).ToList();   // Max degree of parallelism.

for (int a = 0; a < groupedIndexes.Count; a++)
{
    paragraphs.Add(new TextBlock(groupedIndexes[a].Select(i => lines[i]).ToList()));
}

#### Plot paragraphs

In [38]:
var graphs = new List<Graph.Scatter>();
graphs.Add(new Graph.Scatter()
{
    x = new[] { 0, 0, width, width, 0 },
    y = new[] { 0, height, height, 0, 0 },
    mode = "line",
    name = "",
    text = "page",
    marker = new Graph.Marker() { color = "brown" }
});

foreach (var p in paragraphs)
{
    graphs.Add(new Graph.Scatter()
    {
        x = new[] 
        { 
            p.BoundingBox.BottomLeft.X, 
            p.BoundingBox.BottomRight.X, 
            p.BoundingBox.TopRight.X, 
            p.BoundingBox.TopLeft.X,
            p.BoundingBox.BottomLeft.X 
        },
        y = new[]
        { 
            p.BoundingBox.BottomLeft.Y,
            p.BoundingBox.BottomRight.Y,
            p.BoundingBox.TopRight.Y,
            p.BoundingBox.TopLeft.Y,
            p.BoundingBox.BottomLeft.Y
        },
        mode = "lines",
        text = p.Text.Count() <= 25 ? p.Text : string.Join("", p.Text.Take(25)) + "...",
        marker = new Graph.Marker() { color = "red" }
    });
}

var chart = Chart.Plot(graphs.ToArray());
chart.WithLayout(new Layout.Layout() 
                 { 
                     title = pdfName + " - Paragraphs",
                     showlegend = false,
                     hovermode = "closest",
                     width = width,
                     height = height,
                     xaxis = new Graph.Xaxis() {  range = new[] { 0, width } },
                     yaxis = new Graph.Yaxis() {  range = new[] { 0, height } },
                 });
chart.WithXTitle("X");
chart.WithYTitle("Y");
chart.Width = (int)width;
chart.Height = (int)height;
display(chart);